In [1]:
import pandas as pd
import numpy as np
import string
import json

In [2]:
## Static Config
primary_key = " Code"
locations = [
    {
        "column": "District",
        "name": "level1"
    },
    {
        "column": "Municipality/Rural Municipality",
        "name": "level2"
    },
    {
        "column": "Ward",
        "name": "level3"
    }
]

latlong = ["latitude", "longitude"]

charts = []
'''
charts = [
    {
        "name": "Type of Water Points",
        "column": "Is the water point currently functional?",
        "action": "count",
        "type": "stack",
        "value": [
            'Functional in use',
            'Non-functional'
        ],
        "and": None,
        "and_value": None,
    }
]
'''

## Table config
table = []
'''
table = [
    {
        "name": "Summary Data",
        "type": "summary",
        "indicators": [
            {
                "name": "Sub-County",
                "column": "Kebele",
                "action": "distinct",
                "type": "string",
                "value": None,
                "and": None,
                "and_value": None,
            },
            {
                "name": "Total Villages",
                "column": "Village",
                "action": "group-count",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None
            },
            {
                "name": "Number of Functional Waterpoints",
                "column": "Is the water point currently functional?",
                "action": "group-count",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None,
            }
        ],
    }
]
'''

## Marker Detail
marker = {
    "name" : "Water Supply System (WSS) Name",
    "color": [],
    "detail": [
        {
            "name": "Year Commisioned",
            "column": "Construction Year",
            "action": "select",
            "type": "number",
            "value": None,
            "and": None,
            "and_value": None,
        },
        {
            "name": "Source Type",
            "column": "Source Type",
            "action": "select",
            "type": "string",
            "value": None,
            "and": None,
            "and_value": None,
        }
    ]
}

'''
main = {
    'name':'Non-Functional Facility',
    'column': 'Water point Name',
    'select': {
        'column': 'Is the water point currently functional?',
        'value': 'Non-functional'
    },
    'indicators': [{
        'name': 'Number of HHs',
        'column': 'Estimate the number of households who use this water point as their source of water'
    },{
        'name': 'Type of Use',
        'column': 'What is the main purpose for which people use water from this water point?'
    },{
        'name': 'Nature of Technical Breakdown',
        'column': 'Please explain any other observable technical problems that require repairs'
    },{
        'name': 'WUC/WSC Exists',
        'column': 'Is there a WUC/ WSC for this water point?'
    },{
        'name': '% of Community Paying Tariffs',
        'column': '% of Community Paying Tariffs'
    }]
}
'''

"\nmain = {\n    'name':'Non-Functional Facility',\n    'column': 'Water point Name',\n    'select': {\n        'column': 'Is the water point currently functional?',\n        'value': 'Non-functional'\n    },\n    'indicators': [{\n        'name': 'Number of HHs',\n        'column': 'Estimate the number of households who use this water point as their source of water'\n    },{\n        'name': 'Type of Use',\n        'column': 'What is the main purpose for which people use water from this water point?'\n    },{\n        'name': 'Nature of Technical Breakdown',\n        'column': 'Please explain any other observable technical problems that require repairs'\n    },{\n        'name': 'WUC/WSC Exists',\n        'column': 'Is there a WUC/ WSC for this water point?'\n    },{\n        'name': '% of Community Paying Tariffs',\n        'column': '% of Community Paying Tariffs'\n    }]\n}\n"

In [3]:
df = pd.read_csv('nep_wp_data.csv')

In [4]:
# Remove duplicated values
#df = df[~df.duplicated(subset=[' Code'], keep="last")].reset_index()
#df = df.drop(columns=['index'])

In [5]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [6]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num
df['Yield'] = 0

In [7]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [8]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [9]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

primary_key_config = "";
marker_config = {}
latlong_config = {}
name_config = {}
for key in configs:
    if(configs[key].lower() == primary_key.lower()):
        primary_key_config = key
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
    if(configs[key].lower() == marker['name'].lower()):
        marker_config = { "name" : key}
        
#### Locations
loc_config = {}
for loc in locations:
    column_index = ""
    for key in configs:
        if loc['column'].lower() == configs[key].lower():
            column_index = key
    loc_config.update({loc['name'].lower(): column_index})
    
#### Marker Color
marker_color = [];
for mk in marker['color']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_color.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value'],
        "color": mk['color']
    })
marker_config['color'] = marker_color


#### Marker Detail
marker_detail = [];
for mk in marker['detail']:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == mk['column'].lower():
            column = key
        if mk['and'] != None and configs[key].lower() == mk['and'].lower():
            andColumn = key
    marker_detail.append({
        "name": mk['name'],
        "column": column,
        "action": mk['action'],
        "type": mk['type'],
        "value": mk['value'],
        "and": andColumn,
        "and_value": mk['and_value']
    })
marker_config['detail'] = marker_detail
    
    
#### Chart
chart_config = []
for ch in charts:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == ch['column'].lower():
            column = key
        if ch['and'] != None and configs[key].lower() == ch['and'].lower():
            andColumn = key
    chart_config.append({
        "name": ch['name'],
        "column": column,
        "action": ch['action'],
        "type": ch['type'],
        "value": ch['value'],
        "and": andColumn,
        "and_value": ch['and_value'],
    })


#### Table
table_config = []
for tb in table:
    tb_indicators = []
    for ind in tb['indicators']:
        column = None
        andColumn = None;
        for key in configs:
            if configs[key].lower() == ind["column"].lower():
                column = key
            if ind["and"] != None and configs[key].lower() == ind['and'].lower():
                andColumn = key
        tb_indicators.append({ 
            "column": column, 
            "name": ind['name'], 
            "action": ind['action'], 
            "type": ind['type'],
            "value": ind['value'],
            "and": andColumn,
            "and_value": ind['and_value'],
        })
    table_config.append({
        "name": tb['name'],
        "type": tb['type'],
        "indicators": tb_indicators
    })
    
## Main Table
'''
main_config = {'name':main['name']}
main_selector = {}
indicators = []
for key in configs:
    if configs[key] == main['column']:
        main_config.update({'key':key})
    if configs[key] == main['select']['column']:
        main_selector.update({'key':key,'value':main['select']['value']})
    for i in main['indicators']:
        if configs[key] == i['column']:
            indicators.append({'name':i['name'],'key':key})
main_config.update({'indicators':indicators,'select':main_selector})
'''

configs['primary_key'] = primary_key_config
configs['marker'] = marker_config 
configs['charts'] = chart_config
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config
#configs['main'] = main_config

In [10]:
### Replace Dataset Columns
df.columns = index

In [11]:
configs

{'A': ' Code',
 'B': 'District',
 'C': 'Municipality/Rural Municipality',
 'D': 'Ward',
 'E': 'Water Supply System (WSS) Name',
 'F': 'WSS Type',
 'G': 'Latitude',
 'H': 'Longitude',
 'I': 'Elevation',
 'J': 'Construction Year',
 'K': 'Construction Agency',
 'L': 'Supporting Agency',
 'M': 'Source Type',
 'N': 'Source Registration',
 'O': 'Source Registered year',
 'P': 'Source Lat',
 'Q': 'Source Lon',
 'R': 'Source Elv',
 'S': 'Source Intake Type',
 'T': 'Source Intake Condition',
 'U': 'Source Present Water Use',
 'V': 'Source Measured Discharge (lps)',
 'W': 'Source Measured Date',
 'X': 'Source Minimum Yield (lps)',
 'Y': 'Source Water Quality',
 'Z': 'Source Flow Regularity',
 'AA': 'Source Protection',
 'AB': 'Source Treatment Facilities',
 'AC': 'WSUC Fiscal Year',
 'AD': 'WSUC Community Name',
 'AE': 'WSUC Community Formation Date',
 'AF': 'WSUC Address',
 'AG': 'WSUC Phone No',
 'AH': 'WSUC Added Date',
 'AI': 'WSUC WUA Structure',
 'AJ': 'WSUC Meeting',
 'AK': 'WSUC Annual G

In [12]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 61005023,
  'B': 'Surkhet',
  'C': 'Bheriganga',
  'D': 5,
  'E': 'Chamere Water Supply System',
  'F': 'Gravity Pipe System',
  'G': 28.4023701,
  'H': 81.72027059999999,
  'I': 760,
  'J': '2042',
  'K': 'LOCAL',
  'L': 'LOCAL',
  'M': 'Spring',
  'N': 'Source registered & no obstruction',
  'O': 2042,
  'P': 28,
  'Q': 81,
  'R': 762,
  'S': 'Spring Intake',
  'T': 'Good',
  'U': 'Water Supply',
  'V': 1,
  'W': '2018-06-13',
  'X': 1,
  'Y': 'Appropriate treatment facility exists and working',
  'Z': 'Whole year',
  'AA': 'Safe in all respect',
  'AB': 'None',
  'AC': '2074/75',
  'AD': 'Chamere Water Supply System',
  'AE': 0,
  'AF': '',
  'AG': 0,
  'AH': '2018-06-18',
  'AI': 'No Data',
  'AJ': 'No Data',
  'AK': 'Yes ( )',
  'AL': 'Yes',
  'AM': 'No',
  'AN': 'No',
  'AO': 'No',
  'AP': 'No',
  'AQ': 'No Data',
  'AR': '1 No',
  'AS': 'No',
  'AT': 'No Data',
  'AU': 'More than 50%',
  'AV': 0},
 {'A': 61005025,
  'B': 'Surkhet',
  'C': 'Bheriganga',
  'D': 4,
  'E': 'H

In [13]:
### to json
df.to_json('../resources/js/data/nep_wp_data.json', orient="records", indent=2)

with open('../resources/js/data/nep_wp_config.json', 'w') as outfile:
    json.dump(configs, outfile)